# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import nltk
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import os
import re
from scipy.stats import gmean
import pickle

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix,classification_report,fbeta_score, make_scorer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier


In [2]:
database_filepath = "disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)

In [3]:
df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
df['related']=df['related'].map(lambda x: 0 if x == 2 else x)

In [6]:
df = df.drop(['child_alone'],axis=1)

In [8]:
df['related'].unique()

array([1, 0])

In [9]:
# load data from database
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, "urlplaceholder")

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
pipeline0 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline1 = Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer()),
                ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fit = pipeline0.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_prediction_train = pipeline_fit.predict(X_train)
y_prediction_test = pipeline_fit.predict(X_test)
#print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

In [21]:
print(classification_report((y_test.values), (y_prediction_test), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5005
               request       0.75      0.53      0.62      1129
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.59      0.66      2687
          medical_help       0.56      0.26      0.35       526
      medical_products       0.64      0.32      0.43       338
     search_and_rescue       0.57      0.18      0.28       187
              security       0.19      0.03      0.05       135
              military       0.53      0.27      0.36       200
                 water       0.72      0.66      0.69       424
                  food       0.80      0.69      0.74       723
               shelter       0.75      0.55      0.63       565
              clothing       0.70      0.43      0.53       100
                 money       0.49      0.24      0.32       147
        missing_people       0.59      

/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline1.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'count_vectorizer', 'tfidf_transformer', 'classifier', 'count_vectorizer__analyzer', 'count_vectorizer__binary', 'count_vectorizer__decode_error', 'count_vectorizer__dtype', 'count_vectorizer__encoding', 'count_vectorizer__input', 'count_vectorizer__lowercase', 'count_vectorizer__max_df', 'count_vectorizer__max_features', 'count_vectorizer__min_df', 'count_vectorizer__ngram_range', 'count_vectorizer__preprocessor', 'count_vectorizer__stop_words', 'count_vectorizer__strip_accents', 'count_vectorizer__token_pattern', 'count_vectorizer__tokenizer', 'count_vectorizer__vocabulary', 'tfidf_transformer__norm', 'tfidf_transformer__smooth_idf', 'tfidf_transformer__sublinear_tf', 'tfidf_transformer__use_idf', 'classifier__estimator__algorithm', 'classifier__estimator__base_estimator', 'classifier__estimator__learning_rate', 'classifier__estimator__n_estimators', 'classifier__estimator__random_state', 'classifier__estimator', 'classifier__n_jobs'])

In [23]:
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}


cv = GridSearchCV(pipeline1, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('count_vectorizer',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f3b16e17f70>)),
                                       ('tfidf_transformer',
                                        TfidfTransformer()),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__estimator__n_estimators': [10, 20, 40]},
             scoring='f1_micro')

In [25]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      5005
               request       0.58      0.37      0.45      1129
                 offer       0.00      0.00      0.00        24
           aid_related       0.79      0.19      0.31      2687
          medical_help       0.59      0.11      0.19       526
      medical_products       0.68      0.12      0.20       338
     search_and_rescue       0.60      0.17      0.26       187
              security       0.00      0.00      0.00       135
              military       0.33      0.01      0.01       200
                 water       0.58      0.84      0.69       424
                  food       0.77      0.68      0.72       723
               shelter       0.80      0.30      0.44       565
              clothing       1.00      0.01      0.02       100
                 money       0.43      0.14      0.21       147
        missing_people       0.69      

/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print('\n',classification_report(y_train.values, y_prediction_train, target_names=Y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.86     14901
               request       0.62      0.39      0.48      3345
                 offer       1.00      0.04      0.08        94
           aid_related       0.81      0.19      0.31      8173
          medical_help       0.65      0.12      0.20      1558
      medical_products       0.74      0.13      0.22       975
     search_and_rescue       0.64      0.20      0.30       537
              security       0.00      0.00      0.00       336
              military       0.25      0.00      0.00       660
                 water       0.58      0.85      0.69      1248
                  food       0.77      0.68      0.72      2200
               shelter       0.86      0.29      0.44      1749
              clothing       0.50      0.00      0.01       305
                 money       0.60      0.17      0.26       457
        missing_people       0.66    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
#cv.best_params_

#Use pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      4965
               request       0.77      0.56      0.64      1082
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.60      0.67      2631
          medical_help       0.63      0.31      0.41       524
      medical_products       0.65      0.32      0.43       321
     search_and_rescue       0.56      0.19      0.28       185
              security       0.31      0.03      0.06       126
              military       0.57      0.27      0.37       219
                 water       0.74      0.65      0.69       386
                  food       0.82      0.69      0.75       709
               shelter       0.79      0.58      0.67       548
              clothing       0.55      0.37      0.44        86
                 money       0.59      0.30      0.40       158
        missing_people       0.69      

/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mostafahaggag/Data_science_enviroment/data_science_enviroment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [32]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.